In [ ]:
import os
import numpy as np
import spark_dsg as dsg
from spark_dsg.mp3d import load_mp3d_info, repartition_rooms, add_gt_room_label
from hydra_gnn.utils import plot_heterogeneous_graph
from hydra_gnn.preprocess_dsgs import convert_label_to_y, add_object_connectivity, get_room_object_dsg

## Load DSG

In [ ]:
# test dataset file paths
test_json_file = "./test_data/x8F5xyUWy9e_0_gt_partial_dsg_1447.json"
gt_house_file = "./test_data/x8F5xyUWy9e.house"

assert os.path.exists(test_json_file)
assert os.path.exists(gt_house_file)


In [ ]:
# Load hydra scene graph
G = dsg.DynamicSceneGraph.load(test_json_file)
print("Number of nodes separated by layer: {} ({} total).".format([layer.num_nodes() for layer in G.layers], G.num_nodes()))
gt_house_info = load_mp3d_info(gt_house_file)


In [ ]:
data = G.to_torch(use_heterogeneous=True)
fig = plot_heterogeneous_graph(data)
fig.update_layout(
    scene = dict(
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis =dict(visible=False)
        )
    )
    
fig.show()

## Add room label from GT

In [ ]:
# room labels are based on maximum IoU between hydra room bbox and mp3d room segmentation
dsg.add_bounding_boxes_to_layer(G, dsg.DsgLayers.ROOMS)
add_gt_room_label(G, gt_house_info, angle_deg=-90)

## Get room-object graph and convert to torch_data

In [ ]:
# extract room-object dsg and add object connectivity
G_ro = get_room_object_dsg(G, verbose=False)
add_object_connectivity(G_ro, threshold_near=2.0, threshold_on=1.0, max_near=2.0)

In [ ]:
# convert to torch data
def tmp_node_converter(G, x):
    if x.layer == 2:
        return np.hstack((x.attributes.position, x.attributes.bounding_box.max - x.attributes.bounding_box.min, np.zeros(300)))
    else:
        return np.hstack((x.attributes.position, x.attributes.bounding_box.max - x.attributes.bounding_box.min))

data_homogeneous = G_ro.to_torch(use_heterogeneous=False)
data_heterogeneous = G_ro.to_torch(use_heterogeneous=True, node_converter=tmp_node_converter)

In [ ]:
fig = plot_heterogeneous_graph(data_heterogeneous)
fig.show()

In [ ]:
# get label index data_*.y from hydra node labels
synonym_objects = []
synonym_rooms = [('a', 't'), ('z', 'Z', 'x', 'p', '\x15')] 

object_label_dict, room_label_dict = convert_label_to_y(data_homogeneous, object_synonyms=synonym_objects, room_synonyms=synonym_rooms)
object_label_dict, room_label_dict = convert_label_to_y(data_heterogeneous, object_synonyms=synonym_objects, room_synonyms=synonym_rooms)


## Debug

In [ ]:
# hydra labels
labels = [data_heterogeneous[node_type].label for node_type in data_heterogeneous.node_types]
print('object mp3d labels:', labels[0].tolist())
print('room mp3d labels:', [chr(l) for l in labels[1].tolist()])

In [ ]:
# training labels
print(data_heterogeneous['objects'].y)
print(data_heterogeneous['rooms'].y)

In [ ]:
print(data_heterogeneous['objects'].x.shape)
print(data_heterogeneous['rooms'].x.shape)